## Predict AIGramp text with Cloud TPUs and Keras

### Download data

Download textfiles from AIGramp. You use snippets from this file as the *training data* for the model. The *target* snippet is offset by one character.

In [11]:
!wget --show-progress  -O /content/merged_HorrorOld_King_MiscOld_Sheldon.txt http://aigramp.com/texts/merged_HorrorOld_King_MiscOld_Sheldon.txt

--2019-03-17 15:46:59--  http://aigramp.com/texts/merged_HorrorOld_King_MiscOld_Sheldon.txt
Resolving aigramp.com (aigramp.com)... 68.183.102.88
Connecting to aigramp.com (aigramp.com)|68.183.102.88|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30185815 (29M) [text/plain]
Saving to: ‘/content/merged_HorrorOld_King_MiscOld_Sheldon.txt’

/content/merged_Hor 100%[===================>]  28.79M  55.9MB/s    in 0.5s    

2019-03-17 15:47:00 (55.9 MB/s) - ‘/content/merged_HorrorOld_King_MiscOld_Sheldon.txt’ saved [30185815/30185815]



  imports

In [0]:
import sys
import numpy as np
import six
import tensorflow as tf
import time
import os


### Build the data generator

In [27]:
# This address identifies the TPU we'll use when configuring TensorFlow.
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']

THE_TEXT = '/content/merged_HorrorOld_King_MiscOld_Sheldon.txt'

tf.logging.set_verbosity(tf.logging.INFO)

def transform(txt, pad_to=None):
  # drop any non-ascii characters
  output = np.asarray([ord(c) for c in txt if ord(c) < 255], dtype=np.int32)
  if pad_to is not None:
    output = output[:pad_to]
    output = np.concatenate([
        np.zeros([pad_to - len(txt)], dtype=np.int32),
        output,
    ])
  return output

def training_generator(seq_len=100, batch_size=1024):
  """A generator yields (source, target) arrays for training."""
  #with tf.gfile.GFile(THE_TEXT, 'r') as f:
  #  txt = f.read()

  
  with open(THE_TEXT,'rb') as f:
    txt=f.read().decode('utf8',errors='ignore')

    
  tf.logging.info('Input text [%d] %s', len(txt), txt[:50])
  source = transform(txt)
  while True:
    offsets = np.random.randint(0, len(source) - seq_len, batch_size)

    # Our model uses sparse crossentropy loss, but Keras requires labels
    # to have the same rank as the input logits.  We add an empty final
    # dimension to account for this.
    yield (
        np.stack([source[idx:idx + seq_len] for idx in offsets]),
        np.expand_dims(
            np.stack([source[idx + 1:idx + seq_len + 1] for idx in offsets]),
            -1),
    )

six.next(training_generator(seq_len=10, batch_size=1))

INFO:tensorflow:Input text [29838374] ﻿DRACULA




CHAPTER I

JONATHAN HARKER'S J


(array([[101, 114,  46,  32,  84, 111, 119,  97, 114, 100]], dtype=int32),
 array([[[114],
         [ 46],
         [ 32],
         [ 84],
         [111],
         [119],
         [ 97],
         [114],
         [100],
         [ 32]]], dtype=int32))

### Build the model

The model is defined as a two-layer, forward-LSTM—with two changes from the `tf.keras` standard LSTM definition:

1. Define the input `shape` of the model to comply with the [XLA compiler](https://www.tensorflow.org/performance/xla/)'s static shape requirement.
2. Use `tf.train.Optimizer` instead of a standard Keras optimizer (Keras optimizer support is still experimental).

In [0]:
EMBEDDING_DIM = 512

def lstm_model(seq_len=100, batch_size=None, stateful=True):
  """Language model: predict the next word given the current word."""
  source = tf.keras.Input(
      name='seed', shape=(seq_len,), batch_size=batch_size, dtype=tf.int32)

  embedding = tf.keras.layers.Embedding(input_dim=256, output_dim=EMBEDDING_DIM)(source)
  lstm_1 = tf.keras.layers.LSTM(EMBEDDING_DIM, stateful=stateful, return_sequences=True)(embedding)
  lstm_2 = tf.keras.layers.LSTM(EMBEDDING_DIM, stateful=stateful, return_sequences=True)(lstm_1)
  predicted_char = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(256, activation='softmax'))(lstm_2)
  model = tf.keras.Model(inputs=[source], outputs=[predicted_char])
  model.compile(
      optimizer=tf.train.RMSPropOptimizer(learning_rate=0.01),
      loss='sparse_categorical_crossentropy',
      metrics=['sparse_categorical_accuracy'])
  return model

### Train the model

The `tf.contrib.tpu.keras_to_tpu_model` function converts a `tf.keras` model to an equivalent TPU version. You then use the standard Keras methods to train: `fit`, `predict`, and `evaluate`.

In [25]:
tf.keras.backend.clear_session()

training_model = lstm_model(seq_len=100, batch_size=128, stateful=False)

tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    training_model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))

tpu_model.fit_generator(
    training_generator(seq_len=100, batch_size=1024),
    steps_per_epoch=100,
    epochs=10,
)
tpu_model.save_weights('/tmp/bard.h5', overwrite=True)

INFO:tensorflow:Querying Tensorflow master (grpc://10.115.178.42:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 14549445918936634448)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 6030851550365923546)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 12768138642102055941)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 5412330435490471237)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 12337448542735240812)
INFO:tensorflow:*** Available Device: _DeviceAttribute

### Make predictions with the model

Use the trained model to make predictions and generate your own Shakespeare-esque play.
Start the model off with a *seed* sentence, then generate 250 characters from it. The model makes five predictions from the initial seed.

In [26]:
BATCH_SIZE = 50
PREDICT_LEN = 2000

# Keras requires the batch size be specified ahead of time for stateful models.
# We use a sequence length of 1, as we will be feeding in one character at a 
# time and predicting the next character.
prediction_model = lstm_model(seq_len=1, batch_size=BATCH_SIZE, stateful=True)
prediction_model.load_weights('/tmp/bard.h5')

# We seed the model with our initial string, copied BATCH_SIZE times

seed_txt = 'Tom bought an apartment and a car'
seed = transform(seed_txt)
seed = np.repeat(np.expand_dims(seed, 0), BATCH_SIZE, axis=0)

# First, run the seed forward to prime the state of the model.
prediction_model.reset_states()
for i in range(len(seed_txt) - 1):
  prediction_model.predict(seed[:, i:i + 1])

# Now we can accumulate predictions!
predictions = [seed[:, -1:]]
for i in range(PREDICT_LEN):
  last_word = predictions[-1]
  next_probits = prediction_model.predict(last_word)[:, 0, :]
  
  # sample from our output distribution
  next_idx = [
      np.random.choice(256, p=next_probits[i])
      for i in range(BATCH_SIZE)
  ]
  predictions.append(np.asarray(next_idx, dtype=np.int32))
  

for i in range(BATCH_SIZE):
  print('Generated text %d\n\n:' % i)
  p = [predictions[j][i] for j in range(PREDICT_LEN)]
  generated = ''.join([chr(c) for c in p])
  print(generated)
  print()
  assert len(generated) == PREDICT_LEN, 'Generated text too short'

Generated text 0

:
rs thereafter. Up on the far end (each were unknown that the smell of little future is speak-pit back his late. He seemed to get out of town, but the town Princess must be waiting for the dead meeting-house, friends-nothing was driven, that they're going to give a changedmuch and I want to better feed to do is to-circupalous, Herbert asked.The fellow made re-lucks when she felt a stern trotty-old grass'redy river.Mehanade's relationship with the terrible twin, she was strengthing north, and the crawled hand turned the bikes. It was blessed, dresses to his sunplug, then turned a larn in that shade to hand the driver or fingerprints that the two huge man was it behind it was by grandfather's wallet with the Richie with her cheek. The events they all devoted west and determined.'Yeah,' he said quietly.
'I've heard a bad mistake." There was something out from being catch up.'Cox had given him, but I had no small, beautive.' The boy stared at her clicker. She tees since 